### Imports

In [18]:
import numpy as np
import random
# import to do training
from tpg.trainer import Trainer
# import to run an agent (always needed)
from tpg.agent import Agent
# faster training
import multiprocessing as mp

### Grid World

In [19]:
# based (for now) on Downing's Figure 4.1
class GridWorldv1:
    def __init__(self, rows, cols, win_state, starting_state):  
        self.rows = rows
        self.cols = cols
        self.starting_state = starting_state
        self.win_state = win_state
        self.current_state = self.starting_state

    def sample_action (self):
        rand = random.uniform(0, 1)
        if (rand >= 0) and (rand < 0.25):
            return 0
        elif (rand >= 0.25) and (rand < 0.5):
            return 1
        elif (rand >= 0.5) and (rand < 0.75):
            return 2
        else:
            return 3
        
    def reset (self):
        self.current_state = self.starting_state
        return self.current_state
        
    def check_win (self):
        if self.current_state == self.win_state:
            return True
        return False

    # just reset for now...
    def close (self):
        self.current_state = self.starting_state
        return 1
    
    def step (self, action):
        # north
        if action == 0:
            next = (self.current_state[0] + 1, self.current_state[1])
        # south
        elif action == 1:
            next = (self.current_state[0] - 1, self.current_state[1])
        # east
        elif action == 2:
            next = (self.current_state[0], self.current_state[1] + 1)
        # west
        else:
            next = (self.current_state[0], self.current_state[1] - 1)

        terminate = False
        reward = 0
        # check if move is legal
        if (next[0] >= 0 and next[0] <= 9) and (next[1] >= 0 and next[1] <= 9):            
            illegal = 0
            for i in range(5):
                if next == (5, i):
                    illegal = 1
                    
            if (illegal == 0):
                self.current_state = next
                reward -= 0.01
                #print(self.current_state)
            else:
                #print('illegal move!')
                reward -= 1
                #terminate = True
        else:
            #print('out of bounds!')
            reward -= 1
            #terminate = True
        
        if self.check_win():
            reward += 100
            terminate = True
        
        return self.current_state, reward, terminate

### GridWorld Test

In [20]:
env = GridWorldv1(10, 10, (9, 0), (0, 0))
state = env.reset()
#action = agent.act(state)
action = env.sample_action()
state, reward, isDone = env.step(action)
print(state)
print(reward)
print(isDone)

(0, 0)
-1
False


### TPG Agent

In [21]:
def runAgent(args):
    agent = args[0]
    envName = args[1]
    scoreList = args[2]
    numEpisodes = args[3] # number of times to repeat game
    numFrames = args[4] 
    
    # skip if task already done by agent
    if agent.taskDone(envName):
        print('Agent #' + str(agent.agentNum) + ' can skip.')
        scoreList.append((agent.team.id, agent.team.outcomes))
        return
    
    env = GridWorldv1(10, 10, (9, 0), (0, 0))
    valActs = 4
    
    limit = 100000000
    
    # create environment object from above
    scoreTotal = 0
    for ep in range(numEpisodes):
        state = env.reset()
        scoreEp = 0
        states = []
        if numEpisodes > 1:
            numRandFrames = random.randint(0,30)
            limitReached = 0
            for i in range(numFrames): # frame loop
#                 if i < numRandFrames:
#                     env.step(env.sample_action())
#                     continue
                    
                # action selection
                act = agent.act(state)
                
                state, reward, isDone = env.step(act)
                states.append(state)
                #print('State: ' + str(state))
                scoreEp += reward
                
                if isDone:
                    print('Win!')
                    print(states)
                
                if i > limit:
                    print('Limit Reached!')
                    limitReached = True
                
                if isDone or limitReached:
                    break

            print('Agent #' + str(agent.agentNum) + 
              ' | Ep #' + str(ep) + ' | Score: ' + str(scoreEp))
    
        scoreTotal += scoreEp
    scoreTotal /= numEpisodes
    env.close()
    agent.reward(scoreTotal, envName)
    scoreList.append((agent.team.id, agent.team.outcomes))

### All Together Now!

In [22]:
acts = 4
# pAtomic equal to 1 so teams are single nodes
    # starting off, teams don't reference teams
trainer = Trainer(actions=acts, teamPopSize=10, pActAtom=1.0)

envName = 'GridWorldv1'
allScores = []
for gen in range(100):
    scoreList = []

    agents = trainer.getAgents()

    for agent in agents:
        runAgent([agent, envName, scoreList, 10, 1000])

    # apply scores, must do this when multiprocessing
    # because agents can't refer to trainer
    teams = trainer.applyScores(scoreList)

    trainer.evolve(tasks=[envName]) # go into next gen

    # an easier way to track stats than the above example
    scoreStats = trainer.fitnessStats

    allScores.append((scoreStats['min'], scoreStats['max'], scoreStats['average']))

    print('Gen: ' + str(gen))
    print('Results so far: ' + str(allScores))

print('Results:\nMin, Max, Avg')
for score in allScores:
    print(score[0],score[1],score[2])

Agent #0 | Ep #0 | Score: -994.06
Agent #0 | Ep #1 | Score: -1000
Agent #0 | Ep #2 | Score: -1000
Agent #0 | Ep #3 | Score: -1000
Agent #0 | Ep #4 | Score: -1000
Agent #0 | Ep #5 | Score: -1000
Agent #0 | Ep #6 | Score: -1000
Agent #0 | Ep #7 | Score: -1000
Agent #0 | Ep #8 | Score: -1000
Agent #0 | Ep #9 | Score: -1000
Agent #1 | Ep #0 | Score: -1000
Agent #1 | Ep #1 | Score: -1000
Agent #1 | Ep #2 | Score: -1000
Agent #1 | Ep #3 | Score: -1000
Agent #1 | Ep #4 | Score: -1000
Agent #1 | Ep #5 | Score: -1000
Agent #1 | Ep #6 | Score: -1000
Agent #1 | Ep #7 | Score: -1000
Agent #1 | Ep #8 | Score: -1000
Agent #1 | Ep #9 | Score: -1000
Agent #2 | Ep #0 | Score: -996.04
Agent #2 | Ep #1 | Score: -996.04
Agent #2 | Ep #2 | Score: -996.04
Agent #2 | Ep #3 | Score: -996.04
Agent #2 | Ep #4 | Score: -996.04
Agent #2 | Ep #5 | Score: -996.04
Agent #2 | Ep #6 | Score: -996.04
Agent #2 | Ep #7 | Score: -996.04
Agent #2 | Ep #8 | Score: -996.04
Agent #2 | Ep #9 | Score: -996.04
Agent #3 | Ep #0 |

Agent #5 | Ep #1 | Score: -1000
Agent #5 | Ep #2 | Score: -1000
Agent #5 | Ep #3 | Score: -1000
Agent #5 | Ep #4 | Score: -1000
Agent #5 | Ep #5 | Score: -1000
Agent #5 | Ep #6 | Score: -1000
Agent #5 | Ep #7 | Score: -1000
Agent #5 | Ep #8 | Score: -1000
Agent #5 | Ep #9 | Score: -1000
Agent #6 | Ep #0 | Score: -991.09
Agent #6 | Ep #1 | Score: -991.09
Agent #6 | Ep #2 | Score: -991.09
Agent #6 | Ep #3 | Score: -991.09
Agent #6 | Ep #4 | Score: -991.09
Agent #6 | Ep #5 | Score: -991.09
Agent #6 | Ep #6 | Score: -991.09
Agent #6 | Ep #7 | Score: -991.09
Agent #6 | Ep #8 | Score: -991.09
Agent #6 | Ep #9 | Score: -991.09
Agent #7 | Ep #0 | Score: -9.999999999999831
Agent #7 | Ep #1 | Score: -9.999999999999831
Agent #7 | Ep #2 | Score: -9.999999999999831
Agent #7 | Ep #3 | Score: -9.999999999999831
Agent #7 | Ep #4 | Score: -9.999999999999831
Agent #7 | Ep #5 | Score: -9.999999999999831
Agent #7 | Ep #6 | Score: -9.999999999999831
Agent #7 | Ep #7 | Score: -9.999999999999831
Agent #7 | E

Agent #7 | Ep #0 | Score: -991.09
Agent #7 | Ep #1 | Score: -987.13
Agent #7 | Ep #2 | Score: -987.13
Agent #7 | Ep #3 | Score: -987.13
Agent #7 | Ep #4 | Score: -987.13
Agent #7 | Ep #5 | Score: -987.13
Agent #7 | Ep #6 | Score: -987.13
Agent #7 | Ep #7 | Score: -987.13
Agent #7 | Ep #8 | Score: -987.13
Agent #7 | Ep #9 | Score: -987.13
Agent #8 | Ep #0 | Score: -991.09
Agent #8 | Ep #1 | Score: -991.09
Agent #8 | Ep #2 | Score: -991.09
Agent #8 | Ep #3 | Score: -991.09
Agent #8 | Ep #4 | Score: -991.09
Agent #8 | Ep #5 | Score: -991.09
Agent #8 | Ep #6 | Score: -991.09
Agent #8 | Ep #7 | Score: -991.09
Agent #8 | Ep #8 | Score: -991.09
Agent #8 | Ep #9 | Score: -991.09
Agent #9 | Ep #0 | Score: -1000
Agent #9 | Ep #1 | Score: -1000
Agent #9 | Ep #2 | Score: -1000
Agent #9 | Ep #3 | Score: -1000
Agent #9 | Ep #4 | Score: -1000
Agent #9 | Ep #5 | Score: -1000
Agent #9 | Ep #6 | Score: -1000
Agent #9 | Ep #7 | Score: -1000
Agent #9 | Ep #8 | Score: -1000
Agent #9 | Ep #9 | Score: -1000


Agent #6 | Ep #1 | Score: -1000
Agent #6 | Ep #2 | Score: -1000
Agent #6 | Ep #3 | Score: -1000
Agent #6 | Ep #4 | Score: -1000
Agent #6 | Ep #5 | Score: -1000
Agent #6 | Ep #6 | Score: -1000
Agent #6 | Ep #7 | Score: -1000
Agent #6 | Ep #8 | Score: -1000
Agent #6 | Ep #9 | Score: -1000
Agent #7 | Ep #0 | Score: -1000
Agent #7 | Ep #1 | Score: -1000
Agent #7 | Ep #2 | Score: -1000
Agent #7 | Ep #3 | Score: -1000
Agent #7 | Ep #4 | Score: -1000
Agent #7 | Ep #5 | Score: -1000
Agent #7 | Ep #6 | Score: -1000
Agent #7 | Ep #7 | Score: -1000
Agent #7 | Ep #8 | Score: -1000
Agent #7 | Ep #9 | Score: -1000
Agent #8 | Ep #0 | Score: -1000
Agent #8 | Ep #1 | Score: -1000
Agent #8 | Ep #2 | Score: -1000
Agent #8 | Ep #3 | Score: -1000
Agent #8 | Ep #4 | Score: -1000
Agent #8 | Ep #5 | Score: -1000
Agent #8 | Ep #6 | Score: -1000
Agent #8 | Ep #7 | Score: -1000
Agent #8 | Ep #8 | Score: -1000
Agent #8 | Ep #9 | Score: -1000
Agent #9 | Ep #0 | Score: -488.16999999999996
Agent #9 | Ep #1 | Score: 

Agent #5 | Ep #2 | Score: -1000
Agent #5 | Ep #3 | Score: -1000
Agent #5 | Ep #4 | Score: -1000
Agent #5 | Ep #5 | Score: -1000
Agent #5 | Ep #6 | Score: -1000
Agent #5 | Ep #7 | Score: -1000
Agent #5 | Ep #8 | Score: -1000
Agent #5 | Ep #9 | Score: -1000
Agent #6 | Ep #0 | Score: -999.01
Agent #6 | Ep #1 | Score: -1000
Agent #6 | Ep #2 | Score: -1000
Agent #6 | Ep #3 | Score: -1000
Agent #6 | Ep #4 | Score: -1000
Agent #6 | Ep #5 | Score: -1000
Agent #6 | Ep #6 | Score: -1000
Agent #6 | Ep #7 | Score: -1000
Agent #6 | Ep #8 | Score: -1000
Agent #6 | Ep #9 | Score: -1000
Agent #7 | Ep #0 | Score: -1000
Agent #7 | Ep #1 | Score: -1000
Agent #7 | Ep #2 | Score: -1000
Agent #7 | Ep #3 | Score: -1000
Agent #7 | Ep #4 | Score: -1000
Agent #7 | Ep #5 | Score: -1000
Agent #7 | Ep #6 | Score: -1000
Agent #7 | Ep #7 | Score: -1000
Agent #7 | Ep #8 | Score: -1000
Agent #7 | Ep #9 | Score: -1000
Agent #8 | Ep #0 | Score: -1000
Agent #8 | Ep #1 | Score: -1000
Agent #8 | Ep #2 | Score: -1000
Agent 

...

Agent #5 | Ep #5 | Score: -1000
Agent #5 | Ep #6 | Score: -1000
Agent #5 | Ep #7 | Score: -1000
Agent #5 | Ep #8 | Score: -1000
Agent #5 | Ep #9 | Score: -1000
Agent #6 | Ep #0 | Score: -996.04
Agent #6 | Ep #1 | Score: -996.04
Agent #6 | Ep #2 | Score: -996.04
Agent #6 | Ep #3 | Score: -996.04
Agent #6 | Ep #4 | Score: -996.04
Agent #6 | Ep #5 | Score: -996.04
Agent #6 | Ep #6 | Score: -996.04
Agent #6 | Ep #7 | Score: -996.04
Agent #6 | Ep #8 | Score: -996.04
Agent #6 | Ep #9 | Score: -996.04
Agent #7 | Ep #0 | Score: -996.04
Agent #7 | Ep #1 | Score: -996.04
Agent #7 | Ep #2 | Score: -996.04
Agent #7 | Ep #3 | Score: -996.04
Agent #7 | Ep #4 | Score: -996.04
Agent #7 | Ep #5 | Score: -996.04
Agent #7 | Ep #6 | Score: -996.04
Agent #7 | Ep #7 | Score: -996.04
Agent #7 | Ep #8 | Score: -996.04
Agent #7 | Ep #9 | Score: -996.04
Agent #8 | Ep #0 | Score: -996.04
Agent #8 | Ep #1 | Score: -996.04
Agent #8 | Ep #2 | Score: -996.04
Agent #8 | Ep #3 | Score: -996.04
Agent #8 | Ep #4 | Score

Agent #7 | Ep #8 | Score: -9.999999999999831
Agent #7 | Ep #9 | Score: -9.999999999999831
Agent #8 | Ep #0 | Score: -1000
Agent #8 | Ep #1 | Score: -1000
Agent #8 | Ep #2 | Score: -1000
Agent #8 | Ep #3 | Score: -1000
Agent #8 | Ep #4 | Score: -1000
Agent #8 | Ep #5 | Score: -1000
Agent #8 | Ep #6 | Score: -1000
Agent #8 | Ep #7 | Score: -1000
Agent #8 | Ep #8 | Score: -1000
Agent #8 | Ep #9 | Score: -1000
Agent #9 | Ep #0 | Score: -9.999999999999831
Agent #9 | Ep #1 | Score: -9.999999999999831
Agent #9 | Ep #2 | Score: -9.999999999999831
Agent #9 | Ep #3 | Score: -9.999999999999831
Agent #9 | Ep #4 | Score: -9.999999999999831
Agent #9 | Ep #5 | Score: -9.999999999999831
Agent #9 | Ep #6 | Score: -9.999999999999831
Agent #9 | Ep #7 | Score: -9.999999999999831
Agent #9 | Ep #8 | Score: -9.999999999999831
Agent #9 | Ep #9 | Score: -9.999999999999831
Adding dce5ceff-3a99-4932-bdee-522255802a43 to trainer learners
Adding da474ecd-284f-4298-8905-48dd3e072d83 to trainer learners
Adding 3a65a

Agent #5 | Ep #1 | Score: -1000
Agent #5 | Ep #2 | Score: -1000
Agent #5 | Ep #3 | Score: -1000
Agent #5 | Ep #4 | Score: -1000
Agent #5 | Ep #5 | Score: -1000
Agent #5 | Ep #6 | Score: -1000
Agent #5 | Ep #7 | Score: -1000
Agent #5 | Ep #8 | Score: -1000
Agent #5 | Ep #9 | Score: -1000
Agent #6 | Ep #0 | Score: -1000
Agent #6 | Ep #1 | Score: -1000
Agent #6 | Ep #2 | Score: -1000
Agent #6 | Ep #3 | Score: -1000
Agent #6 | Ep #4 | Score: -1000
Agent #6 | Ep #5 | Score: -1000
Agent #6 | Ep #6 | Score: -1000
Agent #6 | Ep #7 | Score: -1000
Agent #6 | Ep #8 | Score: -1000
Agent #6 | Ep #9 | Score: -1000
Agent #7 | Ep #0 | Score: -994.06
Agent #7 | Ep #1 | Score: -996.04
Agent #7 | Ep #2 | Score: -996.04
Agent #7 | Ep #3 | Score: -996.04
Agent #7 | Ep #4 | Score: -996.04
Agent #7 | Ep #5 | Score: -996.04
Agent #7 | Ep #6 | Score: -996.04
Agent #7 | Ep #7 | Score: -996.04
Agent #7 | Ep #8 | Score: -996.04
Agent #7 | Ep #9 | Score: -996.04
Agent #8 | Ep #0 | Score: -493.11999999999995
Agent 

Agent #8 | Ep #8 | Score: -996.04
Agent #8 | Ep #9 | Score: -996.04
Agent #9 | Ep #0 | Score: -489.15999999999997
Agent #9 | Ep #1 | Score: -33.7600000000009
Agent #9 | Ep #2 | Score: -9.999999999999831
Agent #9 | Ep #3 | Score: -9.999999999999831
Agent #9 | Ep #4 | Score: -9.999999999999831
Agent #9 | Ep #5 | Score: -9.999999999999831
Agent #9 | Ep #6 | Score: -9.999999999999831
Agent #9 | Ep #7 | Score: -9.999999999999831
Agent #9 | Ep #8 | Score: -9.999999999999831
Agent #9 | Ep #9 | Score: -9.999999999999831
Adding f37d4b7f-02d0-4636-948f-d2b02539d2c7 to trainer learners
Adding 4e491bcb-32be-4824-826a-960e3de0af81 to trainer learners
Adding 4495dd79-5918-4251-9d01-beb20bfdefef to trainer learners
Adding 40d80fdd-5906-468d-bd74-fd91564f5d07 to trainer learners
Adding da25de4b-e110-4697-a719-7e2430d9e34c to trainer learners
Adding 7d3a8faf-0849-4f1f-bb85-5b94dd92b140 to trainer learners
Adding 6b0db855-107d-4c9f-8d32-758ace4f065f to trainer learners
Adding 99a30f42-f085-4a10-8b6f-30b

Agent #5 | Ep #1 | Score: -9.999999999999831
Agent #5 | Ep #2 | Score: -9.999999999999831
Agent #5 | Ep #3 | Score: -9.999999999999831
Agent #5 | Ep #4 | Score: -9.999999999999831
Agent #5 | Ep #5 | Score: -9.999999999999831
Agent #5 | Ep #6 | Score: -9.999999999999831
Agent #5 | Ep #7 | Score: -9.999999999999831
Agent #5 | Ep #8 | Score: -9.999999999999831
Agent #5 | Ep #9 | Score: -9.999999999999831
Agent #6 | Ep #0 | Score: -989.11
Agent #6 | Ep #1 | Score: -982.1800000000001
Agent #6 | Ep #2 | Score: -982.1800000000001
Agent #6 | Ep #3 | Score: -996.04
Agent #6 | Ep #4 | Score: -996.04
Agent #6 | Ep #5 | Score: -996.04
Agent #6 | Ep #6 | Score: -996.04
Agent #6 | Ep #7 | Score: -996.04
Agent #6 | Ep #8 | Score: -996.04
Agent #6 | Ep #9 | Score: -996.04
Agent #7 | Ep #0 | Score: -989.11
Agent #7 | Ep #1 | Score: -987.13
Agent #7 | Ep #2 | Score: -987.13
Agent #7 | Ep #3 | Score: -987.13
Agent #7 | Ep #4 | Score: -987.13
Agent #7 | Ep #5 | Score: -987.13
Agent #7 | Ep #6 | Score: -98